In [237]:
import mne 
import numpy as np
import os

In [238]:
def arrayFiles(path):
    folderContent = os.listdir(path)
    fifFiles = []
    for i in folderContent :
        fifFiles.append(path + "/" + i)
    
    return fifFiles

In [239]:
#concatenate path to name of each file and store in array for reading in data
path = '/Volumes/flash64/EpochedEEG'
allFifFiles = arrayFiles(path)


In [240]:
epoch = mne.read_epochs(allFifFiles[0],preload=True)

Reading /Volumes/flash64/EpochedEEG/20131216_1441_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     500.00 ms
        0 CTF compensation matrices available
5424 matching events found
Applying baseline correction (mode: mean)
5424 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [241]:
#see all event ids
epoch.event_id

{'congruent': 133,
 'correct': 129,
 'f_baby': 155,
 'f_bell': 156,
 'f_bird': 157,
 'f_burp': 158,
 'f_dog': 159,
 'f_drum': 160,
 'f_knock': 161,
 'f_laugh': 162,
 'f_phone': 163,
 'f_train': 164,
 'f_water': 165,
 'flanker_stim_1': 1,
 'flanker_stim_10': 10,
 'flanker_stim_11': 11,
 'flanker_stim_12': 12,
 'flanker_stim_2': 2,
 'flanker_stim_3': 3,
 'flanker_stim_4': 4,
 'flanker_stim_5': 5,
 'flanker_stim_6': 6,
 'flanker_stim_7': 7,
 'flanker_stim_8': 8,
 'flanker_stim_9': 9,
 'incongruent': 134,
 'incorrect': 130,
 'left_button': 131,
 'right_button': 132,
 't_baby': 135,
 't_bell': 136,
 't_bird': 137,
 't_burp': 138,
 't_dog': 139,
 't_drum': 140,
 't_knock': 141,
 't_laugh': 142,
 't_phone': 143,
 't_train': 144,
 't_water': 145,
 'target_stim_13': 13,
 'target_stim_14': 14,
 'target_stim_15': 15,
 'target_stim_16': 16,
 'target_stim_17': 17,
 'target_stim_18': 18,
 'target_stim_19': 19,
 'target_stim_20': 20,
 'target_stim_21': 21,
 'target_stim_22': 22,
 'target_stim_23': 23

In [290]:
#seperate all flanker events
f_baby_events = epoch.events[epoch.events[:,-1] == 155]
f_bell_events = epoch.events[epoch.events[:,-1] == 156]
f_bird_events = epoch.events[epoch.events[:,-1] == 157] 
f_burp_events = epoch.events[epoch.events[:,-1] == 158]
f_dog_events = epoch.events[epoch.events[:,-1] == 159] 
f_drum_events = epoch.events[epoch.events[:,-1] == 160]
f_knock_events = epoch.events[epoch.events[:,-1] == 161]
f_laugh_events = epoch.events[epoch.events[:,-1] == 162]
f_phone_events = epoch.events[epoch.events[:,-1] == 163]
f_train_events = epoch.events[epoch.events[:,-1] == 164]
f_water_events = epoch.events[epoch.events[:,-1] == 165]



In [282]:
epoch.events

array([[  2411,      0,    131],
       [  2580,      0,     31],
       [  2581,     31,    144],
       ...,
       [658880,      0,      1],
       [658931,      0,    131],
       [659013,      0,    134]])

In [243]:
#array flankers


In [276]:
#156 or 157 or 158 or 159 or 160 or 161 or 162 or 163 or 164 or 165

epoch.events[:,-1] == (155 or 156)


array([False, False, False, ..., False, False, False])

In [287]:
hold = epoch.events[epoch.events[:,-1] == 155]

In [291]:
flankerEpochs = np.concatenate((f_baby_events,f_bell_events,f_bird_events,f_burp_events,
                               f_dog_events,f_drum_events,f_knock_events,f_laugh_events,
                               f_phone_events,f_train_events,f_water_events),axis=0)

In [292]:
#flankerEpochs

array([[181061,    135,    155],
       [182133,    135,    155],
       [183223,      0,    155],
       ...,
       [548211,      0,    165],
       [548990,    137,    165],
       [550734,    145,    165]])

In [293]:
t_baby_events = epoch.events[epoch.events[:,-1] == 135]
t_bell_events = epoch.events[epoch.events[:,-1] == 136]
t_bird_events = epoch.events[epoch.events[:,-1] == 13] 
t_burp_events = epoch.events[epoch.events[:,-1] == 138]
t_dog_events = epoch.events[epoch.events[:,-1] == 139] 
t_drum_events = epoch.events[epoch.events[:,-1] == 140]
t_knock_events = epoch.events[epoch.events[:,-1] == 141]
t_laugh_events = epoch.events[epoch.events[:,-1] == 142]
t_phone_events = epoch.events[epoch.events[:,-1] == 143]
t_train_events = epoch.events[epoch.events[:,-1] == 144]
t_water_events = epoch.events[epoch.events[:,-1] == 145]

In [294]:
targetEpochs = np.concatenate((t_baby_events,t_bell_events,t_bird_events,t_burp_events,
                               t_dog_events,t_drum_events,t_knock_events,t_laugh_events,
                               t_phone_events,t_train_events,t_water_events),axis=0)

In [295]:
#targetEpochs

array([[ 59863,     31,    135],
       [ 61043,     31,    135],
       [ 63464,      0,    135],
       ...,
       [549626,     31,    145],
       [550733,     31,    145],
       [552632,     31,    145]])